In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [15]:
#creating the object to hold to page source content
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
print(page)

#creating an object to store the page source in proper html format
soup=BeautifulSoup(page.content,"html.parser")
#print(soup)

#storing page source belonging to a particular id tag on the page
seven_day=soup.find(id="seven-day-forecast")
print(seven_day)

#finding all instances of a particular class tag within the id and storing it to object
forecast_items=seven_day.find_all(class_="tombstone-container")
print(forecast_items)


tonight=forecast_items[1]
print(tonight.prettify())

<Response [200]>
<div class="panel panel-default" id="seven-day-forecast">
<div class="panel-heading">
<b>Extended Forecast for</b>
<h2 class="panel-title">
	    	    San Francisco CA	</h2>
</div>
<div class="panel-body" id="seven-day-forecast-body">
<div id="seven-day-forecast-container"><div class="current-hazard" id="headline-container" style="margin-left: 124px"><div id="headline-separator" style="top: 34px; height: 131px"></div><div id="headline-info" onclick="$('#headline-detail').toggle(); $('#headline-detail-now').hide()" style="margin-top: 5px"><div id="headline-detail"><div>High Surf Advisory until April 23, 03:00am</div></div><span class="fa fa-info-circle"></span>Click here for hazard details and duration</div><div class="headline-bar headline-advisory" style="top: 40px; left: 19px; height: 125px; width: 74px">
<div class="headline-title">High Surf Advisory</div>
</div></div><ul class="list-unstyled" id="seven-day-forecast-list" style="padding-top: 60px"><li class="forecast

In [16]:
#extracting the values of sub tags
period=tonight.find(class_="period-name").get_text()
short_desc=tonight.find(class_="short-desc").get_text()
temp=tonight.find(class_="temp temp-low").get_text()

print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 48 °F


In [17]:
#extracting the title part of the img tag
img=tonight.find("img")
desc=img["title"]

print(desc)

Tonight: Mostly clear, with a low around 48. West wind 10 to 17 mph, with gusts as high as 23 mph. 


In [18]:
#extracting all values for the period tag from the entire object
period_tags=seven_day.select(".tombstone-container .period-name")
periods=[pt.get_text() for pt in period_tags]
print(periods)

 

#similarly extracting the other tags also from the entire object
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print()
print(temps)
print()
print(descs)

['NOW until3:00am Sat', 'Tonight', 'Saturday', 'SaturdayNight', 'Sunday', 'SundayNight', 'Monday', 'MondayNight', 'Tuesday']
['High Surf Advisory', 'Mostly Clear', 'Mostly Sunnythen Sunnyand Breezy', 'Mostly Clearand Breezythen MostlyClear', 'Sunny', 'Mostly Clear', 'Partly Sunny', 'Partly Cloudyand Breezythen PartlyCloudy', 'Mostly Sunny']

['Low: 48 °F', 'High: 66 °F', 'Low: 50 °F', 'High: 68 °F', 'Low: 50 °F', 'High: 68 °F', 'Low: 51 °F', 'High: 64 °F']

['', 'Tonight: Mostly clear, with a low around 48. West wind 10 to 17 mph, with gusts as high as 23 mph. ', 'Saturday: Sunny, with a high near 66. Breezy, with a west wind 6 to 11 mph increasing to 17 to 22 mph in the afternoon. Winds could gust as high as 29 mph. ', 'Saturday Night: Mostly clear, with a low around 50. Breezy, with a west wind 17 to 22 mph decreasing to 7 to 12 mph in the evening. Winds could gust as high as 28 mph. ', 'Sunday: Sunny, with a high near 68. West wind 5 to 10 mph increasing to 13 to 18 mph in the after

In [19]:
#combining all tags and creating a dataframe out of it
import pandas as pd
weather =pd.DataFrame({
    "period":periods,
     "short_desc":short_descs,
     "temp":["Unknown"]+temps,
    "desc":descs
    
    
})
weather

,period,short_desc,temp,desc
0,NOW until3:00am Sat,High Surf Advisory,Unknown,
1,Tonight,Mostly Clear,Low: 48 °F,"Tonight: Mostly clear, with a low around 48. W..."
2,Saturday,Mostly Sunnythen Sunnyand Breezy,High: 66 °F,"Saturday: Sunny, with a high near 66. Breezy, ..."
3,SaturdayNight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Saturday Night: Mostly clear, with a low aroun..."
4,Sunday,Sunny,High: 68 °F,"Sunday: Sunny, with a high near 68. West wind ..."
5,SundayNight,Mostly Clear,Low: 50 °F,"Sunday Night: Mostly clear, with a low around ..."
6,Monday,Partly Sunny,High: 68 °F,"Monday: Partly sunny, with a high near 68."
7,MondayNight,Partly Cloudyand Breezythen PartlyCloudy,Low: 51 °F,"Monday Night: Partly cloudy, with a low around..."
8,Tuesday,Mostly Sunny,High: 64 °F,"Tuesday: Mostly sunny, with a high near 64."


In [22]:
#desc has the text that contains the value of period also
#trying to split the desc string such that the period is removed
new=weather["desc"].str.split(": ",expand=True)
print(new)

                0                                                  1
0                                                               None
1         Tonight  Mostly clear, with a low around 48. West wind ...
2        Saturday  Sunny, with a high near 66. Breezy, with a wes...
3  Saturday Night  Mostly clear, with a low around 50. Breezy, wi...
4          Sunday  Sunny, with a high near 68. West wind 5 to 10 ...
5    Sunday Night  Mostly clear, with a low around 50. West wind ...
6          Monday                 Partly sunny, with a high near 68.
7    Monday Night      Partly cloudy, with a low around 51. Breezy. 
8         Tuesday                 Mostly sunny, with a high near 64.


In [23]:
#fetching the other part and replacing the desc variable
weather["desc"]=new[1]

In [24]:
#updated dataframe
weather

,period,short_desc,temp,desc
0,NOW until3:00am Sat,High Surf Advisory,Unknown,None
1,Tonight,Mostly Clear,Low: 48 °F,"Mostly clear, with a low around 48. West wind ..."
2,Saturday,Mostly Sunnythen Sunnyand Breezy,High: 66 °F,"Sunny, with a high near 66. Breezy, with a wes..."
3,SaturdayNight,Mostly Clearand Breezythen MostlyClear,Low: 50 °F,"Mostly clear, with a low around 50. Breezy, wi..."
4,Sunday,Sunny,High: 68 °F,"Sunny, with a high near 68. West wind 5 to 10 ..."
5,SundayNight,Mostly Clear,Low: 50 °F,"Mostly clear, with a low around 50. West wind ..."
6,Monday,Partly Sunny,High: 68 °F,"Partly sunny, with a high near 68."
7,MondayNight,Partly Cloudyand Breezythen PartlyCloudy,Low: 51 °F,"Partly cloudy, with a low around 51. Breezy."
8,Tuesday,Mostly Sunny,High: 64 °F,"Mostly sunny, with a high near 64."


In [25]:
weather.to_excel("weather_details_23_04_2022_9:54.xlsx",header=True)